In [ ]:
import base64
import os
from PIL import Image
from io import BytesIO
import requests
import time

In [ ]:
# authentication
engine_id = "stable-diffusion-v1-6"
api_host = os.getenv('API_HOST', 'https://api.stability.ai')
api_key = "sk-dBeXPNuA8GtHabuncaZ6a7uNHnJlnaLVuaz4olhxXOf3pcOn"

In [ ]:
#text to image
def text_to_image(api_key, prompt, debug=False):
    """ generates image based on text prompt and returns image data """
    try:
        response = requests.post(f"{api_host}/v1/generation/{engine_id}/text-to-image",
                                 headers={
                                     "Content-Type": "application/json",
                                     "Accept": "application/json",
                                     "Authorization": f"Bearer {api_key}"},
                                json={
                                    "text_prompts": [
                                        {"text": prompt}
                                    ],
                                    "cfg_scale": 7,
                                    "height": 576,
                                    "width": 1024,
                                    "samples": 1,
                                    "steps": 30,
                                })
        
        data = response.json()
    
        # set debug=true if debugging, will save file to local
        if debug:
            for i, image in enumerate(data["artifacts"]):
                with open(f"./txt2img_{i}.png", "wb") as f:
                    f.write(base64.b64decode(image["base64"]))
        
        image_base64 = response.json()["artifacts"][0]["base64"]  # Assuming only one image is generated
        decoded_image = base64.b64decode(image_base64)
        print(decoded_image)
        return decoded_image

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [ ]:
image_data = text_to_image(api_key, "A forest in a fantasy world with a dragon guarding a cave with mountains in the background")

In [ ]:
# image to video
def image_to_video(api_key, img, debug=False):
    """ generates video based on image and returns video data """
    try:
        get_response = requests.post(
            f"https://api.stability.ai/v2alpha/generation/image-to-video",
            headers={"authorization": api_key},
            files={"image": img},
            data={
                "seed": 0,
                "cfg_scale": 1.8,
                "motion_bucket_id": 127
            },
        )
        
        get_response.raise_for_status()
        generation_id = get_response.json().get('id')
    
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during image-to-video generation: {e}")

    try:
        status = 0
        while status!=200:
            time.sleep(1)
            post_response = requests.request(
            "GET",
            f"https://api.stability.ai/v2alpha/generation/image-to-video/result/{generation_id}",
            headers={
                'Accept': "video/*",  
                'authorization': api_key
            },)
            post_response.raise_for_status()
            status = post_response.status_code
        
        # set debug=true if debugging, will save file to local
        if debug:
            with open("video.mp4", 'wb') as file:
                file.write(post_response.content)
                
        return post_response.content
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during image-to-video result retrieval: {e}")

In [ ]:
video_data = image_to_video(api_key, image_data)